# Programmierübung 1 zu *Grundlagen der Optimierung* (WS2023)

## Einführung

### Verantwortlich
* Dr. Evelyn Herberg
* M.Sc. Masoumeh Hashemi
* B.Sc. Viktor Stein

### Zielsetzung
Das Ziel dieses *Jupyter Notebooks* ist es, Ihnen das Verhalten der im Kapitel 1 des Skripts vorgestellten Algorithmen zur Lösung allgemeiner, unrestringierter Optimierungsprobleme nahezubringen.
Wir werden die Vorteile des *Newton-Verfahrens* gegenüber dem Gradientenverfahren herausarbeiten.

### Zur Nutzung des Notebooks
Die numerische Umsetzung der Verfahren und die graphische Visualisierung typischer Ergebnisse ist ein essentieller Baustein auf dem Weg zu einem ausgereiften Verständnis der Algorithmen.
Um programmiertechnische Schwierigkeiten weitestgehend auszuschließen, haben wir einiges an Code für Sie vorbereitet.
An den Schlüsselstellen der jeweiligen Implementierungen wurde der lauffähige Code durch auskommentierte Blöcke der Art
```python
### TODO BEGIN ###
# Compute the preconditioned gradient and the square of its (preconditioner-induced) norm 
# gradient = ...
# norm2_gradient = ...
### TODO END ###
```
ersetzt, in denen Sie zwischen `### TODO BEGIN ###` und `### TODO END ###` die entsprechenden Anweisungen Variablen (in diesem Beispiel die Berechnung von `gradient` und `norm2_gradient`) entsprechend des Kommentars ausführen, um Lauffähigkeit wieder herzustellen.
Welche Berechnungen und Auswertungen an den jeweiligen Stellen benötigt werden, können Sie im Skript nachlesen.
Sie können natürlich, bevor Sie genau diese vorbenannten Variablen schreiben, auch eigene Variablen beschreiben.

Wenn sie den Code vervollständigt haben, werden Sie an geeigneter Stelle um die Interpretation der Ergebnisse gebeten.
In den entsprechenden Zellen ersetzen Sie "**TODO Ihre Antwort hier**" mit ihrer Antwort.

Notwendige Lösungen aus der ersten Programmieraufgabe werden ihnen im Folgenden bereitgestellt:

In [1]:
# This module implements the preconditioned gradient scheme.

import numpy as np

def gradient_descent(f, x0, step_length_rule, preconditioner, parameters = {}):
  """ 
  Solve an unconstrained minimization problem using the preconditioned
  gradient descent method.

  Accepts:  
                   f: the objective function to be minimized
                  x0: the initial guess (list or numpy array with ndim == 1)
    step_length_rule: a step length computation function 
      preconditioner: a symmetric positive definite matrix (numpy array with ndim == 2)
          parameters: optional parameters (dictionary);
                      the following key/value pairs are evaluated:
                                ["atol_x"]: absolute stopping tolerance for the norm of updates in x
                                ["rtol_x"]: relative stopping tolerance for the norm of updates in x
                                ["atol_f"]: absolute stopping tolerance for the progress in the values of f
                                ["rtol_f"]: relative stopping tolerance for the progress in the values of f
                            ["atol_gradf"]: absolute stopping tolerance for the norm of the gradient of f
                            ["rtol_gradf"]: relative stopping tolerance for the norm of the gradient of f
                        ["max_iterations"]: maximum number of iterations
                             ["verbosity"]: "verbose" or "quiet"
                          ["keep_history"]: whether or not to store the iteration history (True or False) 
                      Here 'norm' refers to the preconditioner-induced norm.
    
  Returns: 
              result: a dictionary containing 
                          solution: final iterate
                          function: the final iterate's objective value 
                          gradient: the final iterate's objective gradient value
                     norm_gradient: preconditioner-induced norm of final objective gradient 
                              iter: number of iterations performed
                          exitflag: flag encoding why the algorithm terminated
                                   0: stopping tolerance described by atol_x, rtol_x, atol_f, rtol_f reached
                                   1: stopping tolerance described by atol_gradf and rtol_gradf reached
                                   2: maximum number of iterations reached

                       history: a dictionary for the history of the run containing
                                          iterates: the iterates x
                                  objective_values: the values of the objective function
                                    gradient_norms: the norms of the objective function gradient 
                                     steps_lengths: the step lengths chosen by the step length rule
  """
  # Define computation of the squared preconditioner norm
  def norm2(d): return d.dot(preconditioner.dot(d))

  # Define an output function that will be used to print information on the state of the iteration
  def print_header(): 
    print('--------------------------------------------------------------------')
    print(' ITER          OBJ    NORM_GRAD    NORM_CORR     OBJ_CHNG           ')
    print('--------------------------------------------------------------------')
  
  # Define exitflags messages that will be printed when the algorithm terminates
  exitflag_messages = [
      'Relative and absolute tolerances on the norm of the update and the descent of the objective are satisfied.',
      'Relative and absolute tolerances on the norm of the gradient are satisfied.',
      'Maximum number of optimization steps is reached.',
      ]
  
  # Get the algorithmic parameters, using defaults if missing
  atol_x = parameters.get("atol_x", 1e-6)
  rtol_x = parameters.get("rtol_x", 1e-6)
  atol_f = parameters.get("atol_f", 1e-6)
  rtol_f = parameters.get("rtol_f", rtol_x**2)
  atol_gradf = parameters.get("atol_gradf", 1e-6)
  rtol_gradf = parameters.get("rtol_gradf", 1e-6)
  max_iterations = parameters.get("max_iterations", 1e3)
  verbosity = parameters.get("verbosity", "quiet")
  keep_history = parameters.get("keep_history", False)

  # Initialize the iterates, counters etc.
  x = x0
  iter = 0
  exitflag = None
  
  # Initialize dummy values pertaining to the previous iterate
  x_old = np.full(x0.shape, np.inf)
  function_value_old = np.inf

  # Prepare a dictionary to store the history
  if keep_history:
    history = {
      "iterates" : [],
      "objective_values" : [],
      "gradient_norms" : [],
      "step_lengths" : []
      }
  
  # Perform gradient descent steps until one of the termination criteria is met
  while exitflag is None:
    # Record the current iterate
    if keep_history: history["iterates"].append(x)
    
    # Dump some output
    if verbosity == 'verbose':
      if (iter%10 == 0): print_header()
      print(' %4d  ' % (iter), end = '')
            
    # Stop when the maximum number of iterations has been reached
    if iter >= max_iterations:
      exitflag = 2
      break
    
    # Compute the function value and derivative at current iterate
    values = f(x, derivatives = [True, True, False])
    function_value = values["function"]
    derivative = values["derivative"]
    
    # Record the current value of the objective
    if keep_history: history["objective_values"].append(function_value)
        
    # Dump some output
    if verbosity == 'verbose': print('%11.4e  ' % (function_value), end = '')
    
    # Compute the preconditioned gradient and the square of its (preconditioner-induced) norm
    gradient = np.linalg.solve(preconditioner, derivative)
    norm2_gradient = derivative.dot(gradient)
    
    # Check the computed norm square for positivity
    if norm2_gradient < 0:
      raise ValueError('Your preconditioner appears not to be positive definite.')
    else:
      norm_gradient = np.sqrt(norm2_gradient)
    
    # Record the current norm of the gradient
    if keep_history: history["gradient_norms"].append(norm_gradient)

    # Remember the norm of the initial gradient
    if (iter == 0): initial_norm_gradient = norm_gradient
        
    # Dump some output
    if verbosity == 'verbose': print('%11.4e  ' % (norm_gradient), end = '')
    
    # Stop when the stopping tolerance on the norm of the gradient has been reached
    if norm_gradient <= atol_gradf + rtol_gradf * initial_norm_gradient:
      exitflag = 1
      break
    
    # Evaluate the norm of the update step
    norm_delta_x = np.sqrt(norm2(x - x_old))
    
    # Evaluate the change in the objective function values
    delta_f = function_value_old - function_value
    
    # Evaluate the reference values for relative tolerances
    abs_function_value_old = np.abs(function_value_old)
    norm_x_old = np.sqrt(norm2(x_old))
    
    # Dump some output
    if verbosity == 'verbose': print('%11.4e  %11.4e' % (norm_delta_x, -delta_f))
    
    # Stop when the stopping tolerance on the change in the objective and the
    # norm of the update step have been reached
    if (delta_f < atol_f + rtol_f * abs_function_value_old) and\
      (norm_delta_x < atol_x + rtol_x * norm_x_old):
      exitflag = 0
      break
    
    # Set the update direction
    d = -gradient
    
    # Prepare the line search function, using the function values of the
    # objective and its derivatives and the chain rule
    def phi(t, derivatives):
      values = f(x + t * d, derivatives)
      if derivatives[1]:
        values["derivative"] = values["derivative"].dot(d)
      if derivatives[2]:
        values["Hessian"] = d.dot(values["Hessian"].dot(d))
      return values
    
    # Prepare some data to pass down to the step length computation rule
    reusables = {
      "phi0" : function_value,
      "dphi0" : -norm2_gradient
      }
    
    # Evaluate the step length t using the step length rule
    t, t_exitflag = step_length_rule(phi, reusables)
    
    # Check whether of not the step length was computed succesfully
    if t_exitflag: raise AssertionError('Step length was not computed succesfully.')
    
    # Record the chosen step length
    if keep_history: history["step_lengths"].append(t)
    
    # Save the current iterate and associated function value for the next iteration
    x_old = x
    function_value_old = function_value
    
    # Update the iterate and increase the counter
    x = x + t * d
    iter = iter + 1

  # Dump some output
  if verbosity == 'verbose':
    print('\n\nThe gradient descent method exiting with flag %d.\n' %(exitflag) + str(exitflag_messages[exitflag])+'\n' )
  
  # Create and populate the result to be returned
  result = {
    "solution" : x,
    "function" : function_value,
    "gradient" : gradient,
    "norm_gradient" : norm_gradient,
    "iter" : iter,
    "exitflag" : exitflag
    }

  # Assign the history to the result if required
  if keep_history:
    result["history"] = history
      
  return result


In [2]:
def armijo_backtracking(phi, reusables = {}, parameters = {}):
  """
  Compute a step length t via backtracking satisfying the Armijo condition 
  for the function phi, i.e.,

    phi(t) <= phi(0) + sigma * t * dphi(0) 

  where dphi(0) is the derivative of phi at t = 0.

  Accepts: 
           phi: evaluates the function the line search is performed on
     reusables: additional information that may be provided to the method (dictionary);
                the following key/value pairs are evaluated:
                   ["phi0"]: the value of phi at t = 0 (scalar)
                  ["dphi0"]: the value of the derivative of phi at t = 0 (scalar)

  Returns:
            t: the step length minimizing phi (provided it is quadratic)
     exitflag: flag encoding why the line search terminated
                 0: success
                 1: maximum number of iterations reached
                 2: trial step length became too small
  """

  def print_header():
    print('--------------------------------------------------------------------')
    print(' ARMIJO:     ITER          STEP      OBJCHNG           ')
    print('--------------------------------------------------------------------')
  
  # Get the line search parameters, using defaults if missing
  sigma = parameters.get("sigma", 0.01)
  beta = parameters.get("beta", 0.5)
  initial_t = parameters.get("initial_step_length", 1.0)
  verbosity = parameters.get("verbosity", "quiet")
  max_iterations = parameters.get("max_iterations", 1e4)
    
  # Extract or compute required data for checking armijo condition
  phi0 = reusables.get("phi0", phi(0, derivatives = [True, False, False])["function"]) or\
    phi(0, derivatives[True,False,False])["function"]
  dphi0 = reusables.get("dphi0", phi(0, derivatives = [False, True, False])["derivative"]) or\
    phi(0, derivatives[False,True,False])["derivative"]
  
  if dphi0 >= 0:
    raise(InputError('The function phi is expected to be decreasing at zero..'))
  
  # Initialize the step length and counter
  t = initial_t
  iter = 0
  exitflag = None
    
  # Perform the backtracking search until one of the termination criteria is met
  while exitflag is None:

    # Evaluate the value of phi at the current trial step length and the amount of descent
    phi_trial = phi(t, derivatives = [True, False, False])["function"]
    delta_phi = phi_trial - phi0
    
    # Dump some output
    if verbosity == 'verbose':
      if (iter%10 == 0): print_header()
      print('             %4d   %11.4e  %11.4e  \n' % (iter, t, delta_phi))
    
    # Verify the Armijo condition
    if delta_phi <= sigma * t * dphi0:
      exitflag = 0
      break
    # Stop when the maximum number of iterations has been reached
    elif iter >= max_iterations:
      exitflag = 1
      print('Warning: Armijo is stopping because the maximum number of iterations is reached.\n')
      break
    # Stop when the function appears locally constant and the initial step length has decreased significantly
    elif (delta_phi == 0) and (t / initial_t < 1e-12): 
      exitflag = 2                               
      if verbosity == 'verbose':
        print('Warning: Armijo is stopping because the function appears locally constant.\n')
      break
    
    # Reduce the trial step size and increase the counter
    t = beta * t
    iter = iter + 1
    
  # Check whether the step length is in fact positive
  if t < 0.0:
    raise ValueError('Armijio is returning a negative step length.')
  else:
    return t, exitflag

## Das globalisierte Newton-Verfahren

Die Effekte, die sie in dem Beispiel der Himmelblaufunktion beobachtet haben zeigen, dass die unreflektierte Verwendung der Hessematrix als "adaptiver Vorkonditionierer" nicht funktionieren kann - daher verwendet man beim Newton-Verfahren die im Skript beschriebene globalisierte Variante.
Dass dieses Verfahren gegenüber dem Gradientenverfahren eine massive Verbesserung der Konvergenzgeschwindigkeit liefern kann, möchten wir anhand der [Rosenbrock Funktion](https://en.wikipedia.org/wiki/Rosenbrock_function) verifizieren.

### Implementierung des globalisierten Newton-Verfahrens
**Aufgabe:** Vervollständigen Sie den Code in der nächsten Zelle und führen Sie die Zelle aus.

In [3]:
import numpy as np

def globalized_newton(f, x0, step_length_rule, preconditioner, parameters = {}):
  """
  Solve an unconstrained minimization problem using the line search globalized newton method.
  
  Accepts:  
                   f: the objective function to be minimized
                  x0: the initial guess (list or numpy array with ndim == 1)
    step_length_rule: a step length computation function 
      preconditioner: a symmetric positive definite matrix (numpy array with ndim == 2)
          parameters: optional parameters (dictionary);
                      the following key/value pairs are evaluated:
                                    ["atol_x"]: absolute stopping tolerance for the norm of updates in x
                                    ["rtol_x"]: relative stopping tolerance for the norm of updates in x
                                    ["atol_f"]: absolute stopping tolerance for the progress in the values of f
                                    ["rtol_f"]: relative stopping tolerance for the progress in the values of f
                                ["atol_gradf"]: absolute stopping tolerance for the norm of the gradient of f
                                ["rtol_gradf"]: relative stopping tolerance for the norm of the gradient of f
                            ["max_iterations"]: maximum number of iterations
                                      ["rho1"]: globalization parameter
                                      ["rho2"]: globalization parameter
                                         ["p"]: globalization parameter
                                 ["verbosity"]: "verbose" or "quiet"
                              ["keep_history"]: whether or not to store the iteration history (True or False) 
                      Here 'norm' refers to the preconditioner-induced norm.
    
  Returns: 
              result: a dictionary containing 
                       solution: final iterate
                       function: the final iterate's objective value 
                       gradient: the final iterate's objective gradient value
                      norm_gradient: preconditioner-induced norm of final objective gradient 
                           iter: number of iterations performed
                       exitflag: flag encoding why the algorithm terminated
                                  0: stopping tolerance described by atol_x, rtol_x, atol_f, rtol_f reached
                                  1: stopping tolerance described by atol_gradf and rtol_gradf reached
                                  2: maximum number of iterations reached

                        history: a dictionary for the history of the run containing
                                              iterates: the iterates x
                                      objective_values: the values of the objective function
                                        gradient_norms: the norms of the objective function gradient 
                                         steps_lengths: the step lengths chosen by the step length rule
                                 used_newton_direction: the information whether or not the newton direction was used
  """
  # Define computation of the squared preconditioner norm
  def norm2(d): return d.dot(preconditioner.dot(d))
  
  # Define an output function that will be used to print information on the state of the iteration
  def print_header():
    print('--------------------------------------------------------------------')
    print(' ITER          OBJ    NORM_GRAD    NORM_CORR     OBJ_CHNG           ')
    print('--------------------------------------------------------------------')
  
  # Define exitflags that will be printed when the algorithm terminates
  exitflag_messages = [
      'Relative and absolute tolerances on the norm of the update and the descent of the objective are satisfied.',
      'Relative and absolute tolerances on the norm of the gradient are satisfied.',
      'Maximum number of optimization steps is reached.',
      ]
  
  # Get the algorithmic parameters, using defaults if missing
  atol_x = parameters.get("atol_x", 1e-6)
  rtol_x = parameters.get("rtol_x", 1e-6)
  atol_f = parameters.get("atol_f", 1e-6)
  rtol_f = parameters.get("rtol_f", rtol_x**2)
  atol_gradf = parameters.get("atol_gradf", 1e-6)
  rtol_gradf = parameters.get("rtol_gradf", 1e-6)
  max_iterations = parameters.get("max_iterations", 1e3)
  rho1 = parameters.get("rho1", 1e-6)
  rho2 = parameters.get("rho2", 1e-6)
  p = parameters.get("p", 0.1)
  verbosity = parameters.get("verbosity", "quiet")
  keep_history = parameters.get("keep_history", False)

  # Initialize the iterates, counters etc.
  x = x0
  iter = 0
  exitflag = None
  used_newton_direction = None
  
  # Initialize dummy values pertaining to the previous iterate
  x_old = np.full(x0.shape, np.inf)
  function_value_old = np.inf

  # Prepare a dictionary to store the history
  if keep_history:
    history = {
      "iterates" : [],
      "objective_values" : [],
      "gradient_norms" : [],
      "step_lengths" : [],
      "used_newton_direction" : []
      }
  
  # Perform gradient descent steps until one of the termination criteria is met
  while exitflag is None:
    # Record the current iterate
    if keep_history: history["iterates"].append(x)
    
    # Dump some output
    if verbosity == 'verbose':
      if (iter%10 == 0): print_header()
      print(' %4d  ' % (iter), end = '')
            
    # Stop when the maximum number of iterations has been reached
    if iter >= max_iterations:
      exitflag = 2
      break
    
    # Compute the function value and derivative at current iterate
    values = f(x, derivatives = [True, True, True])
    function_value = values["function"]
    derivative = values["derivative"]
    hessian = values["Hessian"]
    
    # Record the current value of the objective
    if keep_history: history["objective_values"].append(function_value)
        
    # Dump some output
    if verbosity == 'verbose': print('%11.4e  ' % (function_value), end = '')
    
    ### TODO BEGIN ###
    # Compute the preconditioned gradient and the square of its (preconditioner-induced) norm 
    # gradient = ...
    # norm2_gradient = ...
    ### TODO END ###
    
    # Check the computed norm square for positivity
    if norm2_gradient < 0:
      raise ValueError('Your preconditioner appears not to be positive definite.')
    else:
      norm_gradient = np.sqrt(norm2_gradient)
    
    # Record the current norm of the gradient
    if keep_history: history["gradient_norms"].append(norm_gradient)

    # Remember the norm of the initial gradient
    if (iter == 0): initial_norm_gradient = norm_gradient
        
    # Dump some output
    if verbosity == 'verbose': print('%11.4e  ' % (norm_gradient), end = '')
    
    ### TODO BEGIN ###
    # Stop when the stopping tolerance on the norm of the gradient has been reached
    # if ...
      exitflag = 1
      break
    ### TODO END ###
    
    # Evaluate the norm of the update step
    norm_delta_x = np.sqrt(norm2(x - x_old))
    
    # Evaluate the change in the objective function values
    delta_f = function_value_old - function_value
    
    # Evaluate the reference values for relative tolerances
    abs_function_value_old = np.abs(function_value_old)
    norm_x_old = np.sqrt(norm2(x_old))
    
    # Dump some output
    if verbosity == 'verbose': print('%11.4e  %11.4e' % (norm_delta_x, -delta_f))
    
    # Stop when the stopping tolerance on the change in the objective and the
    # norm of the update step have been reached
    if (delta_f < atol_f + rtol_f * abs_function_value_old) and\
      (norm_delta_x < atol_x + rtol_x * norm_x_old):
      exitflag = 0
      break
    
    ### TODO BEGIN ###
    # Set the update direction
    try:      
      # Compute Newton direction and its (preconditioner induced) norm
      # d = ...
      
      # Check Newton direction for quality
      # if ...
      used_newton_direction = True
    except:
      # Fall back to gradient direction if Newton direction could not be computed or is of poor quality
      used_newton_direction = False
      # d = ...
    ### TODO END ###
    
    # Prepare the line search function, using the function values of the
    # objective and its derivatives and the chain rule
    def phi(t, derivatives):
      values = f(x + t * d, derivatives)
      if derivatives[1]:
        values["derivative"] = values["derivative"].dot(d)
      if derivatives[2]:
        values["Hessian"] = d.dot(values["Hessian"].dot(d))
      return values
    
    # Prepare some data to pass down to the step length computation rule
    reusables = {
      "phi0" : function_value,
      }
    if not used_newton_direction: reusables["dphi0"] = -norm2_gradient
    
    # Evaluate the step length t using the step length rule
    t, t_exitflag = step_length_rule(phi, reusables)
    
    # Check whether of not the step length was computed succesfully
    if t_exitflag: raise AssertionError('Step length was not computed succesfully.')
    
    # Record the chosen step length 
    if keep_history: history["step_lengths"].append(t)
    
    # Save the current iterate and associated function value for the next iteration
    x_old = x
    function_value_old = function_value
    
    ### TODO BEGIN ###
    # Update the iterate and increase the counter
    # x = ...
    # iter = ...
    ### TODO END ###

    # Remember whether or not the newton direction was used as update direction
    if keep_history: history["used_newton_direction"].append(used_newton_direction)

  # Dump some output
  if verbosity == 'verbose':
    print('\n\nThe globalized newton is exiting with flag %d.\n' %(exitflag) + str(exitflag_messages[exitflag])+'\n' )
  
  # Create and populate the result to be returned
  result = {
    "solution" : x,
    "function" : function_value,
    "gradient" : gradient,
    "norm_gradient" : norm_gradient,
    "iter" : iter,
    "exitflag" : exitflag
    }

  # Assign the history to the result if required
  if keep_history:
    result["history"] = history
      
  return result

IndentationError: unexpected indent (1686965772.py, line 148)

Im folgenden Skript wird das globalisierte Newton-Verfahren im Vergleich zum euklidisch vorkonditionierten Gradientenverfahren für die Minimierung der Rosenbrock Funktion verwendet.

**Aufgabe:** Vervollständigen Sie das Skript und führen Sie es aus. Beachten Sie die Bedingungen, die im Skript an die Parameter gestellt werden.

In [ ]:
import sys
sys.path.append('src/')

import numpy as np

from objective_functions import *
from visualization_functions import *

# Create problem data
rosenbrock_parameters = {
    "a" : 1,
    "b" : 10,
  }
f = lambda x, derivatives: rosenbrock(x, derivatives, rosenbrock_parameters)
x0 = np.array([-1.75, 1.5])

### TODO BEGIN ###
# Set parameters for armijo linesearch
# armijo_parameters = {
# "sigma" : ...,
# "beta" : ...,
# "initial_step_length" : ...
#}
### TODO END ###

if not armijo_parameters["sigma"] < 0.5:
  raise ValueError('Chosen armijo sigma is not smaller than 0.5')
if not armijo_parameters["initial_step_length"] == 1:
  raise ValueError('Initial step length is not 1.')

# Construct step length rule
armijo_step_length_rule = lambda phi, reusables: armijo_backtracking(phi, reusables, armijo_parameters);

# Set optimization parameters
optimization_parameters = {
"atol_x" : 1e-7,
"rtol_x" : 1e-7,
"atol_f" : 1e-7,
"rtol_f" : 1e-14,
"max_iterations" : 1e10,
"c" : 10,
"verbosity" : "verbose",
"keep_history" : True
}

# Solve problem using preconditioned gradient scheme
gradient_output = gradient_descent(f, x0, armijo_step_length_rule, preconditioner = np.identity(len(x0)), parameters = optimization_parameters)
gradient_label = "Gradient"
  
# Solve problem using globalized Newton scheme
newton_output = globalized_newton(f, x0, armijo_step_length_rule, preconditioner = np.identity(len(x0)), parameters = optimization_parameters)
newton_label = "Newton"

# Prepare data for plotting
histories = [gradient_output["history"], newton_output["history"]]
labels = [gradient_label, newton_label]

# Plot history in iterate space
plot_2d_iterates_contours(f, histories, labels)

# Plot functional value differences (approximation of error energy norm)
plot_f_val_diffs(histories, 
                list(hist["objective_values"][-1] for hist in histories),
                labels)

plot_step_sizes([newton_output["history"]], [newton_label])

plot_grad_norms(histories, labels)

plot_used_newton_direction([newton_output["history"]], [newton_label])


**Aufgabe:** Beschreiben Sie ihre Beobachtungen zum Konvergenzverhalten der beiden Verfahren.
Warum eignet sich die Rosenbrock Funktion gut für diesen Vergleich? Bringt dem Gradientenverfahren die Wahl anderer Vorkonditionerer einen Vorteil?

**TODO Ihre Antwort hier**

# Bonus

Wenn sie alle Funktionalitäten bis hier entwickeln konnten steht es Ihnen natürlich frei, ihre Lösung zu kopieren und in der Kopie beliebige Änderungen an den Parametern vorzunehmen. 
Eine genauerer Vergleich des Einflusses der Schrittweitenbestimmungen auf das Gradientenverfahren ist bspl. eine weitere interessante Untersuchung, die dem Verständnis der Parameter im Armijo hilfreich ist.